In [138]:
import pandas as pd
import math

In [151]:
######


In [190]:
taxon = pd.read_csv('taxon-product.csv') #item - taxon - product - productLabel
cp = pd.read_csv('companion-planting-wikidata-latin.csv') #name id latin
cp= cp.rename(columns ={'id': 'item'})
cp.item = cp.item.apply(lambda x: f'http://www.wikidata.org/entity/{x}')

In [210]:
for column in cp.columns:
    cp[f'{column}'] = cp[f'{column}'].str.lower()
for column in taxon.columns:
    taxon[f'{column}'] = taxon[f'{column}'].str.lower()

merged = taxon.merge(cp, how = 'outer', on='item')
merged  = merged [['item','taxon','latin','name','product','productLabel']]
def combItem(a,b):
    if a == b:
        return a
    if not pd.isnull(b):
        return b
    return a

merged['taxon']= merged.apply(lambda x: combItem(x.taxon, x.latin) ,axis=1)
result= merged.drop(['latin'],axis=1)[['taxon', 'name', 'item', 'productLabel', 'product']]
result =result.rename(columns= {'name':'plantCommonName','item':'plantWikidata', 'productLabel': 'productCommonName',
                       'product': 'productWikidata'}).drop_duplicates()

### changes in names
diff = {'beans':'bean'}
result.plantCommonName = result.plantCommonName.apply(lambda x: diff[x] if x in diff.keys() else x)

result.to_csv('./names-taxon-products.csv')

In [230]:
r2 = result.copy()
r2.plantCommonName = r2.plantCommonName.apply(lambda x:  x[0:-1])
r2 = pd.concat([result,r2])[['plantCommonName','taxon']]

In [231]:
comp = pd.read_csv('companion-planting.csv', names = ['v1','v2','rel'], sep =';')
comp.v1 = comp.v1.str.lower()
comp.v2 = comp.v2.str.lower()
comp

,v1,v2,rel
0,apple,chives,companion
1,apple,garlic,companion
2,apple,grass,antagonistic
3,apple,horseradish,companion
4,apple,lemon balm,companion
...,...,...,...
539,yarrow,grass,companion
540,yarrow,tansy,companion
541,zucchini,corn,companion
542,zucchini,marjoram,companion


In [232]:
comp1 = comp.merge(r2, how = 'left', left_on = 'v1', right_on = 'plantCommonName')
comp2 = comp1.merge(r2, how = 'left', left_on = 'v2', right_on = 'plantCommonName',suffixes = ('_v1','_v2')).drop_duplicates()

In [235]:
comp2[['v1','taxon_v1','v2','taxon_v2','rel']].drop_duplicates().to_csv('./companion_plants_including_taxon.csv')